In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import requests
import geocoder

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
toronto_neighborhoods = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(toronto_neighborhoods, 'lxml')

In [3]:
data = []
headers = []
table = soup.find('table', class_='wikitable sortable').tbody

rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('th') + row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if row.find_all('th') > row.find_all('td'):
        headers.append([ele for ele in cols if ele])
    else:
        data.append([ele for ele in cols if ele])

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
dataframe = pd.DataFrame(data, columns = headers[0])
# Rename the first & third columns
dataframe.rename(columns={'Postcode':'PostalCode','Neighbourhood':'Neighborhood'}, inplace=True)
dataframe

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
df_all_boroughs = dataframe[dataframe.Borough != 'Not assigned']
df_all_boroughs.reset_index(inplace=True, drop=True)
df_all_boroughs

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [6]:
df_all_boroughs_corrected = df_all_boroughs.copy()
df_all_boroughs_corrected.loc[df_all_boroughs_corrected['Neighborhood'] == 'Not assigned',
                    'Neighborhood'] = df_all_boroughs_corrected['Borough']
df_all_boroughs_corrected.reset_index(inplace=True, drop=True)
df_all_boroughs_corrected

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [7]:
# Output will be a series, so need to convert back to df.
df_ab_grouped = pd.DataFrame(df_all_boroughs_corrected.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join))
df_ab_grouped.reset_index(inplace=True)
df_ab_grouped

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
df_ab_grouped.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [114]:
# Get latitudes and longitudes for all neighborhoods from Arcgis
toronto_neighborhood_lat_lng = {}

rows = df_ab_grouped['PostalCode']
for row in rows:
    lat_lng = geocoder.arcgis('{}, Toronto, Ontario'.format(row)).latlng
    
    # key = PostalCode, lat = [PostalCode][0], long = [PostalCode][1]
    toronto_neighborhood_lat_lng[row] = lat_lng

In [116]:
# Dict with all neighborhood latidudes and longitudes
toronto_neighborhood_lat_lng

{'M1B': [43.811525000000074, -79.19551721399995],
 'M1C': [43.78573000000006, -79.15874999999994],
 'M1E': [43.76569000000006, -79.17525603599995],
 'M1G': [43.76835912100006, -79.21758999999997],
 'M1H': [43.76968799900004, -79.23943999999995],
 'M1J': [43.74312500000008, -79.23174973599998],
 'M1K': [43.726244585000074, -79.26366999999993],
 'M1L': [43.71313321100007, -79.28505499999994],
 'M1M': [43.72357500000004, -79.23497617799995],
 'M1N': [43.69666500000005, -79.26016331599999],
 'M1P': [43.759975000000054, -79.26897402899993],
 'M1R': [43.750710464000065, -79.30055999999996],
 'M1S': [43.79394000000008, -79.26798280099996],
 'M1T': [43.78472500000004, -79.29904659999994],
 'M1V': [43.817810000000065, -79.28024362199994],
 'M1W': [43.80088094900003, -79.32073999999994],
 'M1X': [43.83421500000003, -79.21670085099998],
 'M2H': [43.80284500000005, -79.35623615099996],
 'M2J': [43.780880000000025, -79.34779577599994],
 'M2K': [43.781015000000025, -79.38054242199996],
 'M2L': [43.7

In [117]:
# Append latitudes & longitudes to dataframe
df_ab_grouped_latlng = df_ab_grouped.copy()

df_ab_grouped_latlng['Latitude'] = \
    df_ab_grouped_latlng['PostalCode'].map(pd.DataFrame.from_dict(toronto_neighborhood_lat_lng).transpose()[0])
df_ab_grouped_latlng['Longitude'] = \
    df_ab_grouped_latlng['PostalCode'].map(pd.DataFrame.from_dict(toronto_neighborhood_lat_lng).transpose()[1])

df_ab_grouped_latlng

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163
